In [1]:
# This code is best run inside Google Colab, or some alternations have to be made to the code.

In [2]:
!pip install pdfplumber
from google.colab import files
import json, os, shutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 30.2 MB/s eta 0:00:00


In [4]:
# === Step 1. Upload your Label Studio JSON ===
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving MathewDavis_IntakeLabelStudioExport.json to MathewDavis_IntakeLabelStudioExport.json


In [5]:
# === Step 2. Load the JSON file ===
with open(filename, 'r') as f:
    data = json.load(f)

# Normalize data to a list
if isinstance(data, dict):
    annotations = [data]
elif isinstance(data, list):
    annotations = data
else:
    raise ValueError("Unexpected JSON format — must be list or dict")

print(f"✅ Loaded {len(annotations)} annotations from {filename}")

✅ Loaded 4 annotations from MathewDavis_IntakeLabelStudioExport.json


In [6]:
def schema_to_standard_json(entry):
    output = {
        "patient_name": None,
        "patient_dob": None,
        "patient_phone": None,
        "referral_name": None,
        "provider_name": None
    }


    # --- Detect which format we're working with ---
    if "annotations" in entry:
        results = entry.get("annotations", [])[0].get("result", [])
        labels = [r.get("value", {}) for r in results if "value" in r]
    elif "label" in entry:
        labels = entry["label"]
    else:
        labels = []

    # --- Process all labels uniformly ---
    for label in labels:
        label_type = label.get("labels", [None])[0]
        text = label.get("text", "").strip()

        if label_type and label_type in output:
            output[label_type] = text

    return output

In [12]:
# === Step 4. Create output folder ===
os.makedirs("standardized_intakes", exist_ok=True)

In [13]:
# === Step 5. Convert and save each intake ===
for entry in annotations:
    converted_entry = schema_to_standard_json(entry)

    # Use filename or ID for naming
    base_name = entry.get("filename") or f"intake_{entry.get('id', 'unknown')}.json"
    base_name = os.path.splitext(os.path.basename(base_name))[0] + ".json"

    out_path = os.path.join("standardized_intakes", base_name)
    with open(out_path, "w") as f:
        json.dump(converted_entry, f, indent=2)

print(f"✅ Saved {len(annotations)} JSON files in 'standardized_intakes/'")

✅ Saved 4 JSON files in 'standardized_intakes/'


In [ ]:
# === Step 6. Zip and download all files ===
shutil.make_archive("standardized_intakes", 'zip', "standardized_intakes")
files.download("standardized_intakes.zip")

print("🎉 All done! Your ZIP file is ready for download.")